In [3]:
import pandas as pd
from datetime import datetime
import numpy as np
#import pm4py
from collections import defaultdict
#from Levenshtein import distance

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn import metrics
#from sklearn.metrics.cluster import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, silhouette_score

import copy

In [55]:
import pm4py
log = pm4py.read.read_xes("wabo_all.xes")

C:\Users\la1949\Anaconda3\envs\pm4py\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 4348/4348 [00:22<00:00, 193.45it/s]


In [3]:
"""
# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'tabName_element' column
log['activity'] = le.fit_transform(log['taskName'])
log_select = log[['case:concept:name', 'time:timestamp', 'activity', 'case:sublog']]
log_select2 = log_select.sort_values(by='time:timestamp',ascending=False)
log_select_new = log_select2.drop_duplicates(subset=['case:concept:name'])
log_select_new = log_select_new.sort_values(by='case:concept:name',ascending=True)


####

logVar = log_select2.groupby(['case:concept:name'])['activity'].apply(list).reset_index()


####

#Add clusters to data frame
log_select_new = log_select_new.drop(["time:timestamp", "activity"], axis=1)
logVarClusters = pd.merge(logVar, log_select_new, on="case:concept:name")

logVarClusters['caseID'] = le.fit_transform(logVarClusters['case:concept:name'])
#logVarClusters
"""

In [4]:
#logVarClusters['label'] = le.fit_transform(logVarClusters['case:sublog'])


In [9]:
#logVarClusters['label'].unique()

array([0, 3, 1, 2, 4])

In [8]:
"""#Derive set of unique trace variants + count
logVarClusters["act_str"] = logVarClusters["activity"].apply(lambda x: str(x))
#logVar['Count'] = logVar.groupby(['act_str'])['concept:name'].transform('count')
logVarClusters = logVarClusters.drop_duplicates(subset=['act_str'], keep='first')
logVarClusters = logVarClusters.reset_index(drop=True)"""

,case:concept:name,activity,case:sublog,caseID,label,act_str
0,10358444,"[343, 236, 366, 260, 357, 366, 260, 357, 74, 2...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,0,0,"[343, 236, 366, 260, 357, 366, 260, 357, 74, 2..."
1,11269753,"[343, 366, 366, 260, 260, 357, 357, 445, 201, ...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,1,0,"[343, 366, 366, 260, 260, 357, 357, 445, 201, ..."
2,11666570,"[445, 197, 196, 380, 341, 395, 170, 99, 332, 4...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,2,0,"[445, 197, 196, 380, 341, 395, 170, 99, 332, 4..."
3,12421232,"[208, 260, 28, 199, 357, 204, 5, 141, 164, 212...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,3,0,"[208, 260, 28, 199, 357, 204, 5, 141, 164, 212..."
4,12592056,"[208, 28, 260, 199, 70, 357, 204, 12, 72, 164,...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,4,0,"[208, 28, 260, 199, 70, 357, 204, 12, 72, 164,..."
...,...,...,...,...,...,...
4285,9987434,"[21, 158, 431, 322, 376, 153, 425, 426, 404, 9...",/usr/share/eventlogs/12714542/CoSeLoG WABO 5.xes,4341,2,"[21, 158, 431, 322, 376, 153, 425, 426, 404, 9..."
4286,9988302,"[271, 158, 404, 153, 21, 93, 296, 426, 425, 338]",/usr/share/eventlogs/12714542/CoSeLoG WABO 5.xes,4342,2,"[271, 158, 404, 153, 21, 93, 296, 426, 425, 338]"
4287,9989988,"[271, 153, 425, 426, 21, 93, 296, 338, 158, 404]",/usr/share/eventlogs/12714542/CoSeLoG WABO 5.xes,4343,2,"[271, 153, 425, 426, 21, 93, 296, 338, 158, 404]"
4288,9990823,"[442, 321, 104, 20, 319, 34, 60, 5, 212, 164, ...",/usr/share/eventlogs/12714542/CoSeLoG WABO 5.xes,4344,2,"[442, 321, 104, 20, 319, 34, 60, 5, 212, 164, ..."


In [10]:
"""
n = 70

logVar1 = logVarClusters.loc[logVarClusters['label'] == 0]
logVar1 = logVar1.sample(n)

logVar2 = logVarClusters.loc[logVarClusters['label'] == 1]
logVar2 = logVar2.sample(n)

logVar3 = logVarClusters.loc[logVarClusters['label'] == 2]
logVar3 = logVar3.sample(n)

logVar4 = logVarClusters.loc[logVarClusters['label'] == 3]
logVar4 = logVar4.sample(n)

logVar5 = logVarClusters.loc[logVarClusters['label'] == 4]
logVar5 = logVar5.sample(n)


logVar = pd.concat([logVar1, logVar2, logVar3, logVar4, logVar5]).reset_index()
"""

In [7]:
#logVar.to_csv('logVar_Sample1.csv', index = False)

In [4]:
logVar = pd.read_csv('logVar_Evaluation_Wabo.csv')
logVar

,index,case:concept:name,activity,case:sublog,caseID,label
0,163,20309364,"[222, 445, 197, 395, 170, 196, 341, 380, 99, 3...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,163,0
1,111,20098000,"[343, 196, 341, 197, 445, 380, 222, 170, 395, ...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,111,0
2,2021,4648742,"[445, 343, 236, 201, 99, 366, 208, 260, 28, 19...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,2021,0
3,185,20466285,"[343, 222, 395, 197, 170, 445, 380, 99, 196, 1...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,185,0
4,891,3631277,"[445, 89, 99, 366, 208, 28, 349, 204, 357, 260...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,891,0
...,...,...,...,...,...,...
345,2191,4885164,"[236, 99, 366, 208, 260, 28, 204, 12, 394, 5, ...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,2191,4
346,3331,6376407,"[170, 236, 74, 116, 332, 167, 413, 255, 260, 2...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,3331,4
347,3031,5894090,"[99, 366, 208, 260, 28, 199, 357, 204, 12, 394...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,3031,4
348,2744,5454228,"[445, 99, 236, 366, 208, 260, 28, 199, 204, 35...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,2744,4


In [5]:
logVar['label'].value_counts()

0    70
1    70
2    70
3    70
4    70
Name: label, dtype: int64

In [6]:
import ast

#Calculate Levenshtein Distances
logVar["activity"] = logVar["activity"].apply(lambda x: ast.literal_eval(x))
logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(int(i)) for i in x])
logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [7]:
logVar["length"] = logVar["activity"].apply(lambda x: len(x))

In [8]:
logVar['length'].max()

110

In [9]:
logVar['length'].min()

1

In [10]:
logVar

,index,case:concept:name,activity,case:sublog,caseID,label,c:n_chr,strings,length
0,163,20309364,"[222, 445, 197, 395, 170, 196, 341, 380, 99, 3...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,163,0,"[Þ, ƽ, Å, Ƌ, ª, Ä, ŕ, ż, c, ŗ, Ā, Ą, á, Ō, §, ...","ÞƽÅƋªÄŕżcŗĀĄáŌ§ƞ ©'ŜQ¦ŦÌUó Ɗs""ƺ<Ô¤ŁhņĿÝ...",62
1,111,20098000,"[343, 196, 341, 197, 445, 380, 222, 170, 395, ...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,111,0,"[ŗ, Ä, ŕ, Å, ƽ, ż, Þ, ª, Ƌ, c, ƞ, Ō, §, Ą, Ā, ...","ŗÄŕÅƽżÞªƋcƞŌ§ĄĀ ©'Ŝ¦ŦÌQóUs ƊŁÔƺ""¤<áÝhņĞ...",88
2,2021,4648742,"[445, 343, 236, 201, 99, 366, 208, 260, 28, 19...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,2021,0,"[ƽ, ŗ, ì, É, c, Ů, Ð, Ą, , Ç, Ì, ť, , Ɗ, , ...","ƽŗìÉcŮÐĄ ÇÌť Ɗ<ƺ¤Ô""ŀhĿ9Ŕ$ļĕŸƔĎ]ĨŒ",41
3,185,20466285,"[343, 222, 395, 197, 170, 445, 380, 99, 196, 1...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,185,0,"[ŗ, Þ, Ƌ, Å, ª, ƽ, ż, c, Ä, §, ƞ, Ō, Ŧ, Ł, "", ...","ŗÞƋÅªƽżcÄ§ƞŌŦŁ""QUƊs¦óŜ'ÝĄ¤ ÌÔ© Āá<ƺĞhņ...",85
4,891,3631277,"[445, 89, 99, 366, 208, 28, 349, 204, 357, 260...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,891,0,"[ƽ, Y, c, Ů, Ð, , ŝ, Ì, ť, Ą, , Ɗ, , <, ƺ, ...","ƽYcŮÐ ŝÌťĄ Ɗ<ƺ¤Ô""ƹŖŶŉƸ&ƓńŸĭīĕĎėŒ",43
...,...,...,...,...,...,...,...,...,...
345,2191,4885164,"[236, 99, 366, 208, 260, 28, 204, 12, 394, 5, ...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,2191,4,"[ì, c, Ů, Ð, Ą, , Ì, , Ɗ, , <, ƺ, Ƃ, Ɠ, ť, ...","ìcŮÐĄ Ì Ɗ<ƺƂƓťÔ¤""ƹhĿ9Ƹ&ĞŵĔĺƁĿ9ƸŉļŸĕ...",51
346,3331,6376407,"[170, 236, 74, 116, 332, 167, 413, 255, 260, 2...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,3331,4,"[ª, ì, J, t, Ō, §, Ɲ, ÿ, Ą, , , ©, Ŝ, Ì, Ŧ, ...","ªìJtŌ§ƝÿĄ ©ŜÌŦQU ǏƣvĐ Ŏŋİ¶ŭĲŠĴ±Ï""ƺ¤ÔŁŀhƹ...",70
347,3031,5894090,"[99, 366, 208, 260, 28, 199, 357, 204, 12, 394...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,3031,4,"[c, Ů, Ð, Ą, , Ç, ť, Ì, , Ɗ, , <, ¤, ƺ, Ô, ...","cŮÐĄ ÇťÌ Ɗ<¤ƺÔŀ""hĿ9ŔļĕŸ$Ĩ]Œ",36
348,2744,5454228,"[445, 99, 236, 366, 208, 260, 28, 199, 204, 35...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,2744,4,"[ƽ, c, ì, Ů, Ð, Ą, , Ç, Ì, ť, ŗ, Ɗ, , , <, ...","ƽcìŮÐĄ ÇÌťŗƊ <ƺÔ¤""ŀŔĦhĿ9ƓƂƏĕļŸ$]&Œ",43


## Ground truth 

In [11]:
#Create dictionary with true labels

labelDict1 = logVar['label'].to_dict()

In [12]:
logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(i) for i in x])
logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [13]:
def matrix_calc(features, distance):
    n = len(features)
    dist_matrix = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i, n):
            dist_matrix[i,j] = distance(features[i], features[j])
            dist_matrix[j,i] = dist_matrix[i,j]
    
    return dist_matrix

In [14]:
def results(DistMatrix):
    print('NN:   ' + str(NearestNeighbor(DistMatrix, labelDict1)))
    print('P@10: ' + str(PrecisionAtK(DistMatrix, labelDict1, 10)))

    triplet = BatchAllTtripletLoss()
    print('Tri:  ' + str(triplet.forward(DistMatrix,labelDict1)))
    print('Sil:  ' + str(Silhouette(DistMatrix, labelDict1)))

## Evaluation based on Nearest Neighbor

In [15]:
def NearestNeighbor(matrix, labelDict):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    for i in range(len(matrix)):
        #delete 0 in array matrix[i] for distance between (i,i)
        x = np.delete(matrix[i], i) 
        #identify min distance/value in array
        y = min(x)
        
        #identify position of y AND select first position/pair appearing in the array in case there are muliple pairs with identical min distance
        nearestNeighbor = np.where(x == y)[0] #problem if multiple positions??
        nearestNeighbor = int(nearestNeighbor[0])
        
        #true Label for i
        trueLabel = labelDict[i]
        
        #for label comparison add +1 to dict position if position occurs after i (because of the deletion of 0 at the beginning)
        if nearestNeighbor >= i:
            if trueLabel == labelDict[nearestNeighbor + 1]:
                clusterMeasuredCount[trueLabel] += 1         
        else:
            if trueLabel == labelDict[nearestNeighbor]:
                clusterMeasuredCount[trueLabel] += 1
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterMeasuredCount)
    #print(clusterTrueCount)
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 8,  9,  5, 4, 0]])

labelDict2 = {0:1,1:2,2:1,3:3,4:3}
NearestNeighbor(x, labelDict2)
'''

'\nx = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 8,  9,  5, 4, 0]])\n\nlabelDict2 = {0:1,1:2,2:1,3:3,4:3}\nNearestNeighbor(x, labelDict2)\n'

## Evaluation based on Precision@k

In [16]:
def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    #return None  # Return None if the value is not found in the dictionary




def PrecisionAtK(matrix, labelDict, k):
    
    clusterLabel = set(labelDict.values())
    clusterMeasuredCount = dict.fromkeys(list(clusterLabel), 0)
    
    
    for i in range(len(matrix)):
        
        #true Label for i
        trueLabel = labelDict[i]

        
        #delete 0 in array matrix[i] for distance between (i,i)   
        x = np.delete(matrix[i], i)
        
        #get all k minimum values
        nnValues = []
        for l in range(k):
            y = min(x)
            nnValues.append(y)
            x = np.delete(x, np.where(x == y)[0][0])
        
        
        #get location of minimum values
        z = np.delete(matrix[i], i)
        
        #create dict from array with {position:value}
        my_dict = {}
        for m in range(len(z)):
            my_dict[m] = z[m]
        
        key_list = []
        for n in nnValues:
            key = get_key_by_value(my_dict, n)
            key_list.append(key)
            del my_dict[key]
        
        for o in range(k):
            position = list(key_list)[o]

            if position >= i:
                if trueLabel == labelDict[position + 1]:
                    clusterMeasuredCount[trueLabel] += 1

            else:
                if trueLabel == labelDict[position]:
                    clusterMeasuredCount[trueLabel] += 1

            #transform array to dict
            #get key value from dict
            #compare label
            #remove key+value from dict

            #Need exception in case y == 0 ??
            
    
    #Count the (true) number of traces per label/attribute
    clusterTrueCount = {}
    for i in clusterLabel:
        clusterTrueCount[i] = list(labelDict.values()).count(i)
    
    #Divide number of nearest neighbours with identical label by the respective (true number) of traces with this label
    metric = 0
    for i in clusterLabel:
        metric += clusterMeasuredCount[i] / k / clusterTrueCount[i]
        
    #print(clusterMeasuredCount, clusterTrueCount, metric)
    
    #print(clusterLabel)
    #print(metric)
    return metric / len(clusterLabel)


'''
x3 = np.array([[ 0, 12,  8, 10, 12],
       [5,  0,  9, 14, 17],
       [ 8,  9,  0, 12, 10],
       [ 8,  9,  1, 0, 10],
       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!

labelDict3 = {0:1,1:2,2:1,3:3,4:3}
PrecisionAtK(x3, labelDict3, 2)
'''

'\nx3 = np.array([[ 0, 12,  8, 10, 12],\n       [5,  0,  9, 14, 17],\n       [ 8,  9,  0, 12, 10],\n       [ 8,  9,  1, 0, 10],\n       [ 4,  8,  4, 4, 0]]) # --> Issue: How to select NN when identical distance values !!!\n\nlabelDict3 = {0:1,1:2,2:1,3:3,4:3}\nPrecisionAtK(x3, labelDict3, 2)\n'

## Triplet

see: https://towardsdatascience.com/triplet-loss-advanced-intro-49a07b7d8905

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
def get_triplet_mask(labels):
    
    # step 1 - get a mask for distinct indices
    ###print('labels', labels)
    # shape: (batch_size, batch_size)
    indices_equal = torch.eye(labels.size()[0], dtype=torch.bool, device=labels.device)
    ###print('equal', indices_equal)
    indices_not_equal = torch.logical_not(indices_equal)
    ###print('not_equal', indices_not_equal)

    # shape: (batch_size, batch_size, 1)
    i_not_equal_j = indices_not_equal.unsqueeze(2)
    ###print('i_not_j - unsqueeze2', i_not_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_not_equal_k = indices_not_equal.unsqueeze(1)
    ###print('i_not_k - unsqueeze1', i_not_equal_k)
    # shape: (1, batch_size, batch_size)
    j_not_equal_k = indices_not_equal.unsqueeze(0)
    ###print('j_not_k - unsqueeze0', i_not_equal_k)
    # Shape: (batch_size, batch_size, batch_size)
    distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
    ###print('distinct!!!!', distinct_indices)

    # step 2 - get a mask for valid anchor-positive-negative triplets
    # shape: (batch_size, batch_size)
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)
    ###print('labels_equal', labels_equal)
    # shape: (batch_size, batch_size, 1)
    i_equal_j = labels_equal.unsqueeze(2)
    ###print('i_equal_j', i_equal_j)
    # shape: (batch_size, 1, batch_size)
    i_equal_k = labels_equal.unsqueeze(1)
    ###print('i_equal_k', i_equal_k)
    # shape: (batch_size, batch_size, batch_size)
    valid_indices = torch.logical_and(i_equal_j, torch.logical_not(i_equal_k))
    ###print('valid_indices!!!', valid_indices)
    

    # step 3 - combine two masks
    mask = torch.logical_and(distinct_indices, valid_indices)
    ###print('mask!!', mask)
    return mask

    """compute a mask for valid triplets
    Args:
        labels: Batch of integer labels. shape: (batch_size,)
    Returns:
        Mask tensor to indicate which triplets are actually valid. Shape: (batch_size, batch_size, batch_size)
        A triplet is valid if:
        `labels[i] == labels[j] and labels[i] != labels[k]`
        and `i`, `j`, `k` are different.
    """
    
class custom_activation(nn.Module):
    def __init__(self):
        super(custom_activation, self).__init__()
    
    def forward(self, x):
        x[x>0] = 1
        x[x<=0] = 0
        return x


class BatchAllTtripletLoss(nn.Module):
  """Uses all valid triplets to compute Triplet loss

  Args:
    margin: Margin value in the Triplet Loss equation
  """
  def __init__(self, margin=0): #default margin = 0
    super().__init__()
    self.margin = margin
    self.relu = nn.ReLU() #new
    self.custom = custom_activation()
    
  def forward(self, distance_matrix, labels):
    """computes loss value.

    Args:
      embeddings: Batch of embeddings, e.g., output of the encoder. shape: (batch_size, embedding_dim)
      labels: Batch of integer labels associated with embeddings. shape: (batch_size,)

    Returns:
      Scalar loss value.
    """
    # step 1 - convert to tensor format
    distance_matrix = torch.tensor(distance_matrix)
    labels = torch.tensor(list(labels.values()))


    # step 2 - compute loss values for all triplets by applying broadcasting to distance matrix

    # shape: (batch_size, batch_size, 1)
    anchor_positive_dists = distance_matrix.unsqueeze(2)
    # shape: (batch_size, 1, batch_size)
    anchor_negative_dists = distance_matrix.unsqueeze(1)
    # get loss values for all possible n^3 triplets
    # shape: (batch_size, batch_size, batch_size)
    triplet_loss = anchor_negative_dists - anchor_positive_dists + self.margin
    ###print('tl0',triplet_loss)

    # step 3 - filter out invalid or easy triplets by setting their loss values to 0

    # shape: (batch_size, batch_size, batch_size)
    mask = get_triplet_mask(labels)
    ###print('mask', mask)
    triplet_loss *= mask
    ###print(triplet_loss)
    ###print('tl1:', triplet_loss)
    # easy triplets have negative loss values
    
    triplet_loss = self.custom(triplet_loss)
    ###print(triplet_loss)
    #triplet_loss = F.relu(triplet_loss)

    # step 4 - compute scalar loss value by averaging positive losses
    
    triLossNonZero = (triplet_loss != 0).nonzero(as_tuple=True)
    labelTorchUnique = torch.unique(labels, return_counts=True)
    
    nonZero = len(triLossNonZero[0])
    triLossSum = []
    for i in range(nonZero):
        #Identify L_a --> In Class
        t1 = triLossNonZero[0][i]
        labelIn = labels[t1]
        positionIn = int((labelTorchUnique[0] == labelIn).nonzero(as_tuple=False))
        countIn = labelTorchUnique[1][positionIn]

        #Identify L_b --> Out Class
        t3 = triLossNonZero[2][i]
        labelOut = labels[t3]
        positionOut = int((labelTorchUnique[0] == labelOut).nonzero(as_tuple=False))
        countOut = labelTorchUnique[1][positionOut]

        #Calculate loss
        value = (1/countIn)*(1/countIn)*(1/countOut)  
        ###print(countIn)
        ###print(countOut)
        triLossSum.append(value)
    
    #finally divide by |A|^2-|A|
    A = len(labelTorchUnique[0])  
    lossValue = sum(triLossSum) / (A*A-A)
        
    #OLD
    #E_triplet = (1 / (A^2 - A)) *
    #num_positive_losses = (triplet_loss > eps).float().sum()
    #print(num_positive_losses)
    #print(triplet_loss.sum())
    #triplet_loss = triplet_loss.sum() / (num_positive_losses + eps)
    

    return lossValue

## Silhouette

In [19]:
from sklearn import metrics

def Silhouette(distMatrix, labelDict):
    labelDictList = list(labelDict.values())
    return metrics.silhouette_score(distMatrix, labelDictList)

In [19]:
logVar['activity']

0      [222, 445, 197, 395, 170, 196, 341, 380, 99, 3...
1      [343, 196, 341, 197, 445, 380, 222, 170, 395, ...
2      [445, 343, 236, 201, 99, 366, 208, 260, 28, 19...
3      [343, 222, 395, 197, 170, 445, 380, 99, 196, 1...
4      [445, 89, 99, 366, 208, 28, 349, 204, 357, 260...
                             ...                        
345    [236, 99, 366, 208, 260, 28, 204, 12, 394, 5, ...
346    [170, 236, 74, 116, 332, 167, 413, 255, 260, 2...
347    [99, 366, 208, 260, 28, 199, 357, 204, 12, 394...
348    [445, 99, 236, 366, 208, 260, 28, 199, 204, 35...
349    [445, 343, 99, 236, 208, 28, 199, 204, 394, 12...
Name: activity, Length: 350, dtype: object

### Levenshtein Distance

In [11]:
#import ast
#
##Calculate Levenshtein Distances
#logVar["activity"] = logVar["activity"].apply(lambda x: ast.literal_eval(x))
#logVar["c:n_chr"] = logVar["activity"].apply(lambda x: [chr(int(i)) for i in x])
#logVar["strings"] = logVar["c:n_chr"].apply(lambda x: ''.join(x))

In [12]:
#Levenshtein Distance

List = list(logVar["strings"])

dist_matrix = np.zeros((len(List),len(List)),dtype=int)

for i in range(0,len(List)):
    for j in range(0,len(List)):
        dist_matrix[i,j] = distance(List[i],List[j])

lev_dis = dist_matrix

In [16]:
Matrix = lev_dis

print('NN:   ' + str(NearestNeighbor(Matrix, labelDict1)))
print('P@10: ' + str(PrecisionAtK(Matrix, labelDict1, 10)))
print('Sil:  ' + str(Silhouette(Matrix, labelDict1)))

triplet = BatchAllTtripletLoss()
print('Tri:  ' + str(triplet.forward(Matrix,labelDict1)))


NN:   0.6028571428571429
P@10: 0.398
Sil:  -0.0791734036326659
Tri:  tensor(0.5038)


### Normalized Levenshtein Distance

In [25]:
List = logVar["strings"]

n = len(List)
dist_matrix = np.zeros((n,n))    # initialize distance matrix to a square of zeros

for i in range(n):
    for j in range(i, n):
        dist_matrix[i,j] = distance(List[i], List[j]) / max(len(List[i]),len(List[j]))
        dist_matrix[j,i] = dist_matrix[i,j]       # for the symmetric part, no computation

lev_dis_norm = dist_matrix

In [26]:
Matrix = lev_dis_norm

print('NN:   ' + str(NearestNeighbor(Matrix, labelDict1)))
print('P@10: ' + str(PrecisionAtK(Matrix, labelDict1, 10)))
print('Sil:  ' + str(Silhouette(Matrix, labelDict1)))

triplet = BatchAllTtripletLoss()
print('Tri:  ' + str(triplet.forward(Matrix,labelDict1)))


NN:   0.5999999999999999
P@10: 0.40028571428571436
Sil:  -0.032571737993892784
Tri:  tensor(0.5151)


### Cosine based on 1-gram

In [20]:
#Create 1-gram

def createVector(charList):
    #dtype = [('structure', 'S10'), ('relfrequ', float)]
    arrayList = np.array(charList)
    unique, counts = np.unique(arrayList, return_counts=True)
    #calculate relative frequency
    relFrequList = np.array((unique, counts)).T
    uniqueList = list(unique)
    return relFrequList[relFrequList[:, 0].argsort()]
    #check completeness
    #if 'tree' not in uniqueList:
        #relFrequList = np.append(relFrequList, np.array([['tree', 0]]), axis=0)
        #print(relFrequList)

        
#Change data format from string to list of unique characters
logVar["1-gram"] = logVar["c:n_chr"].apply(lambda x: createVector(tuple(x)))
#logVar


In [21]:
def alignArrays(array1, array2):
    commonSet = set(array1[:,0]).union(array2[:,0])
    
    for i in commonSet:
        if i not in array1[:,0]:
            array1 = np.append(array1, np.array([[i, '0']]), axis=0)
        if i not in array2[:,0]:
            array2 = np.append(array2, np.array([[i, '0']]), axis=0)
    return array1[array1[:, 0].argsort()], array2[array2[:, 0].argsort()]

In [22]:
from scipy.spatial import distance

def cosineDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(int)
    b = Vector2[:,1].astype(int)
    dist_matrix = distance.cosine(a, b)
    return dist_matrix

In [172]:
cos1_dis = matrix_calc(logVar["1-gram"],cosineDist)

In [22]:
#Cosine distance based on 1-gram

cos1_dis = matrix_calc(logVar["1-gram"],cosineDist)
results(cos1_dis)

NN:   0.6428571428571428
P@10: 0.43914285714285717
Tri:  tensor(0.5240)
Sil:  -0.03964055630404477


### Cosine based on 2-gram and 3-gram

In [23]:
#Change data format from string to list of unique characters
#logVar["charList"] = logVar["trace_variant"].apply(lambda x: list(x))
#logVar

def df_list(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList)):
        new = ''.join(extList[i:i+2])
        list_new.append(new)
    del list_new[-1]
    return list_new

#Change data format from string to list of unique characters
logVar["dfList"] = logVar["c:n_chr"].apply(lambda x: df_list(x))
#logVar

logVar["2-gram"] = logVar["dfList"].apply(lambda x: createVector(x))
#logVar

In [24]:
#Cosine distance based on 2-gram
cos2_dis = matrix_calc(logVar["2-gram"],cosineDist)
results(cos2_dis)

NN:   0.6114285714285714
P@10: 0.3934285714285714
Tri:  tensor(0.5258)
Sil:  -0.03499878822714153


In [24]:
#Create 3-gram

#Change data format from string to list of unique characters
logVar["charList"] = logVar["c:n_chr"].apply(lambda x: list(x))

def df_list2(list_of_char):
    extList = list_of_char.copy()
    extList.insert(0, '*') 
    extList.append('$')
    list_new = []
    for i in range(len(extList) - 1):
        new = ''.join(extList[i:i+3])
        list_new.append(new)
    del list_new[-1]
    return list_new

In [25]:
logVar["dfList2"] = logVar["charList"].apply(lambda x: df_list2(x))
logVar["3-gram"] = logVar["dfList2"].apply(lambda x: createVector(x))

In [27]:
#Cosine distance based on 3-gram

cos3_dis = matrix_calc(logVar["3-gram"],cosineDist)
results(cos3_dis)

NN:   0.6085714285714285
P@10: 0.3857142857142857
Tri:  tensor(0.4072)
Sil:  -0.02830259157767106


In [74]:
#Aggregation

aggregate1 = cos1_dis + cos2_dis
results(aggregate1)

NN:   0.6428571428571429
P@10: 0.42371428571428565
Tri:  tensor(0.5293)
Sil:  -0.03555974355713172


## Euclidean Distance

In [28]:
# Euclidean distance
# see https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy

def euclidDist(frequVector1, frequVector2):
    Vector1, Vector2 = alignArrays(frequVector1, frequVector2)
    a = Vector1[:,1].astype(float)
    b = Vector2[:,1].astype(float)
    euclidean_dist = np.linalg.norm(a-b)
    return euclidean_dist

In [29]:
#Euclidean distance based on 1-gram

euc1_dis = matrix_calc(logVar["1-gram"],euclidDist)
results(euc1_dis)

NN:   0.6285714285714286
P@10: 0.43257142857142855
Tri:  tensor(0.5116)
Sil:  -0.040854331452135406


In [30]:
#Euclidean distance based on 2-gram

euc2_dis = matrix_calc(logVar["2-gram"],euclidDist)
results(euc2_dis)

NN:   0.46571428571428575
P@10: 0.3071428571428571
Tri:  tensor(0.5041)
Sil:  -0.059293061201904036


In [31]:
#Euclidean distance based on 3-gram

euc3_dis = matrix_calc(logVar["3-gram"],euclidDist)
results(euc3_dis)

NN:   0.36571428571428577
P@10: 0.2508571428571429
Tri:  tensor(0.4956)
Sil:  -0.06254562165355021


In [32]:
#Aggregation

agg_euc = euc1_dis + euc2_dis
results(agg_euc)

NN:   0.6885714285714286
P@10: 0.4382857142857143
Tri:  tensor(0.5216)
Sil:  -0.04690545504969452


### Jaccard based on 1-gram

In [26]:
#Distance based on activity type

def jaccard_similarity(list1, list2):
    try:
        s1, s2 = set(list1), set(list2)
        return 1 - len(s1 & s2) / len(s1 | s2)
    except ZeroDivisionError:
        return 0

In [34]:
#Jaccard based on 1-gram
Jacc1_dis = matrix_calc(logVar["charList"],jaccard_similarity)
results(Jacc1_dis)

NN:   0.6457142857142857
P@10: 0.43742857142857144
Tri:  tensor(0.5236)
Sil:  -0.03552169941647459


In [35]:
#Jaccard based on 2-gram
Jacc2_dis = matrix_calc(logVar["dfList"],jaccard_similarity)
results(Jacc2_dis)

NN:   0.6114285714285714
P@10: 0.3868571428571429
Tri:  tensor(0.5255)
Sil:  -0.02630847338243588


In [36]:
#Jaccard based on 3-gram
Jacc3_dis = matrix_calc(logVar["dfList2"],jaccard_similarity)
results(Jacc3_dis)

NN:   0.6057142857142856
P@10: 0.37885714285714284
Tri:  tensor(0.4070)
Sil:  -0.015493164917695838


In [37]:
#Aggregation

agg_jacc1 = Jacc1_dis + Jacc2_dis
results(agg_jacc1)

NN:   0.6828571428571428
P@10: 0.44371428571428567
Tri:  tensor(0.5286)
Sil:  -0.03293724249347572


In [38]:
agg_jacc2 = Jacc1_dis + Jacc2_dis + Jacc3_dis
results(agg_jacc2)

NN:   0.6485714285714286
P@10: 0.4388571428571429
Tri:  tensor(0.5294)
Sil:  -0.028774891038859596


## Parallelism based on ordering of activities

In [27]:
logVar

,index,case:concept:name,activity,case:sublog,caseID,label,c:n_chr,strings,length,1-gram,dfList,2-gram,charList,dfList2,3-gram
0,163,20309364,"[222, 445, 197, 395, 170, 196, 341, 380, 99, 3...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,163,0,"[Þ, ƽ, Å, Ƌ, ª, Ä, ŕ, ż, c, ŗ, Ā, Ą, á, Ō, §, ...","ÞƽÅƋªÄŕżcŗĀĄáŌ§ƞ ©'ŜQ¦ŦÌUó Ɗs""ƺ<Ô¤ŁhņĿÝ...",62,"[[, 1], [ , 1], [, 1], [, 1], [, 1], [ , 1...","[*Þ, Þƽ, ƽÅ, ÅƋ, Ƌª, ªÄ, Äŕ, ŕż, żc, cŗ, ŗĀ, Ā...","[["", 1], [ Ɗ, 1], [Ŀ, 1], [], 1], [Ǝ, 1], ...","[Þ, ƽ, Å, Ƌ, ª, Ä, ŕ, ż, c, ŗ, Ā, Ą, á, Ō, §, ...","[*Þƽ, ÞƽÅ, ƽÅƋ, ÅƋª, ƋªÄ, ªÄŕ, Äŕż, ŕżc, żcŗ, ...","[[""ƺ, 1], [ Ɗs, 1], [Ŀ, 1], [], 1], [ƎŸ,..."
1,111,20098000,"[343, 196, 341, 197, 445, 380, 222, 170, 395, ...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,111,0,"[ŗ, Ä, ŕ, Å, ƽ, ż, Þ, ª, Ƌ, c, ƞ, Ō, §, Ą, Ā, ...","ŗÄŕÅƽżÞªƋcƞŌ§ĄĀ ©'Ŝ¦ŦÌQóUs ƊŁÔƺ""¤<áÝhņĞ...",88,"[[, 1], [ , 1], [, 1], [, 2], [, 1], [, 1...","[*ŗ, ŗÄ, Äŕ, ŕÅ, Åƽ, ƽż, żÞ, Þª, ªƋ, Ƌc, cƞ, ƞ...","[[s, 1], [ Ɗ, 1], [ś, 1], [, 1], [Ğ, 1], ...","[ŗ, Ä, ŕ, Å, ƽ, ż, Þ, ª, Ƌ, c, ƞ, Ō, §, Ą, Ā, ...","[*ŗÄ, ŗÄŕ, ÄŕÅ, ŕÅƽ, Åƽż, ƽżÞ, żÞª, ÞªƋ, ªƋc, ...","[[s , 1], [ ƊŁ, 1], [śƓ, 1], [Ŀ, 1], [Ğ,..."
2,2021,4648742,"[445, 343, 236, 201, 99, 366, 208, 260, 28, 19...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,2021,0,"[ƽ, ŗ, ì, É, c, Ů, Ð, Ą, , Ç, Ì, ť, , Ɗ, , ...","ƽŗìÉcŮÐĄ ÇÌť Ɗ<ƺ¤Ô""ŀhĿ9Ŕ$ļĕŸƔĎ]ĨŒ",41,"[[, 1], [ , 1], [, 1], [ , 1], ["", 1], [$, 1...","[*ƽ, ƽŗ, ŗì, ìÉ, Éc, cŮ, ŮÐ, ÐĄ, Ą , Ç, ÇÌ, Ì...","[[<, 1], [ Ɗ, 1], [Ŀ, 1], [ Ç, 1], [""ŀ, 1], ...","[ƽ, ŗ, ì, É, c, Ů, Ð, Ą, , Ç, Ì, ť, , Ɗ, , ...","[*ƽŗ, ƽŗì, ŗìÉ, ìÉc, ÉcŮ, cŮÐ, ŮÐĄ, ÐĄ , Ą Ç, ...","[[<ƺ, 1], [ Ɗ, 1], [Ŀ, 1], [ ÇÌ, 1], [""ŀ,..."
3,185,20466285,"[343, 222, 395, 197, 170, 445, 380, 99, 196, 1...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,185,0,"[ŗ, Þ, Ƌ, Å, ª, ƽ, ż, c, Ä, §, ƞ, Ō, Ŧ, Ł, "", ...","ŗÞƋÅªƽżcÄ§ƞŌŦŁ""QUƊs¦óŜ'ÝĄ¤ ÌÔ© Āá<ƺĞhņ...",85,"[[, 1], [ , 1], [, 1], [, 2], [, 1], [, 1...","[*ŗ, ŗÞ, ÞƋ, ƋÅ, Åª, ªƽ, ƽż, żc, cÄ, Ä§, §ƞ, ƞ...","[[ƺ, 1], [ Ā, 1], [ś, 1], [¡, 1], [Ŀ, 1], ...","[ŗ, Þ, Ƌ, Å, ª, ƽ, ż, c, Ä, §, ƞ, Ō, Ŧ, Ł, "", ...","[*ŗÞ, ŗÞƋ, ÞƋÅ, ƋÅª, Åªƽ, ªƽż, ƽżc, żcÄ, cÄ§, ...","[[ƺĞ, 1], [ Ā, 1], [śƓ, 1], [¡\, 1], [Ŀ,..."
4,891,3631277,"[445, 89, 99, 366, 208, 28, 349, 204, 357, 260...",/usr/share/eventlogs/12688511/CoSeLoG WABO 2.xes,891,0,"[ƽ, Y, c, Ů, Ð, , ŝ, Ì, ť, Ą, , Ɗ, , <, ƺ, ...","ƽYcŮÐ ŝÌťĄ Ɗ<ƺ¤Ô""ƹŖŶŉƸ&ƓńŸĭīĕĎėŒ",43,"[[, 1], [ , 1], [, 1], [, 1], [, 1], [ , 1...","[*ƽ, ƽY, Yc, cŮ, ŮÐ, Ð , ŝ, ŝÌ, Ìť, ťĄ, Ą , ...","[[<, 1], [ Ɗ, 1], [, 1], [&, 1], [ŉ, 1], ...","[ƽ, Y, c, Ů, Ð, , ŝ, Ì, ť, Ą, , Ɗ, , <, ƺ, ...","[*ƽY, ƽYc, YcŮ, cŮÐ, ŮÐ , Ð ŝ, ŝÌ, ŝÌť, ÌťĄ, ...","[[<ƺ, 1], [ Ɗ, 1], [Ŗ, 1], [&Ɠ, 1], [ŉ,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2191,4885164,"[236, 99, 366, 208, 260, 28, 204, 12, 394, 5, ...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,2191,4,"[ì, c, Ů, Ð, Ą, , Ì, , Ɗ, , <, ƺ, Ƃ, Ɠ, ť, ...","ìcŮÐĄ Ì Ɗ<ƺƂƓťÔ¤""ƹhĿ9Ƹ&ĞŵĔĺƁĿ9ƸŉļŸĕ...",51,"[[, 1], [ , 1], [, 2], [, 1], [, 1], [, 1...","[*ì, ìc, cŮ, ŮÐ, ÐĄ, Ą , Ì, Ì , Ɗ, Ɗ, <, <...","[[<, 1], [ Ɗ, 1], [, 1], [Ŀ, 1], [ŉ, 1], ...","[ì, c, Ů, Ð, Ą, , Ì, , Ɗ, , <, ƺ, Ƃ, Ɠ, ť, ...","[*ìc, ìcŮ, cŮÐ, ŮÐĄ, ÐĄ , Ą Ì, Ì , Ì Ɗ, Ɗ, ...","[[<ƺ, 1], [ Ɗ, 1], [Ŀ, 1], [Ŀ, 1], [ŉļ,..."
346,3331,6376407,"[170, 236, 74, 116, 332, 167, 413, 255, 260, 2...",/usr/share/eventlogs/12718076/CoSeLoG WABO 4.xes,3331,4,"[ª, ì, J, t, Ō, §, Ɲ, ÿ, Ą, , , ©, Ŝ, Ì, Ŧ, ...","ªìJtŌ§ƝÿĄ ©ŜÌŦQU ǏƣvĐ Ŏŋİ¶ŭĲŠĴ±Ï""ƺ¤ÔŁŀhƹ...",70,"[[, 1], [ , 1], [, 1], [, 1], [, 1], [, 1...","[*ª, ªì, ìJ, Jt, tŌ, Ō§, §Ɲ, Ɲÿ, ÿĄ, Ą , , ...","[[ , 1], [ Ǐ, 1], [¤, 1], [ŀ, 1], [], 1], ...","[ª, ì, J, t, Ō, §, Ɲ, ÿ, Ą, , , ©, Ŝ, Ì, Ŧ, ...","[*ªì, ªìJ, ìJt, JtŌ, tŌ§, Ō§Ɲ, §Ɲÿ, ƝÿĄ, ÿĄ , ...","[[ Ǐ, 1], [ Ǐƣ, 1], [¤Ô, 1], [ŀh, 1], []Ĩ,..."
347,3031,5894090,"[99, 366, 208, 260, 2

In [28]:
# Get set of unique df-relations

# Step 1: Drop None values (if any)
df_clean = logVar.dropna(subset=['dfList'])

# Step 2: Explode the lists into individual rows
exploded = df_clean.explode('dfList')

# Step 3: Get unique values
unique_values = exploded['dfList'].unique()

# Step 4: Convert to set (optional)
df_relations = set(unique_values)

#df_relations


In [29]:
# Get set of unique 3-grams

# Step 1: Drop None values (if any)
df_clean = logVar.dropna(subset=['dfList2'])

# Step 2: Explode the lists into individual rows
exploded = df_clean.explode('dfList2')

# Step 3: Get unique values
unique_values = exploded['dfList2'].unique()

# Step 4: Convert to set (optional)
gram_relations = set(unique_values)

#gram_relations

In [30]:
def join_and_replace(lst, start, end):
    # Join the specified slice
    joined = ''.join(sorted(lst[start:start+end])) # sort according to alphabet
    # Replace that slice with the joined string
    return lst[:start] + [joined] + lst[start+end:]

def checkPara(traceList,dfSet,gramSet):
    i = 0
    while i < len(traceList) - 1:
        #print('i=',i)
        j = i
        count = 1
        #print('j=',j)
        #print(traceList[j + 1] + traceList[j])
        
        #check whether df-relation exist in opposite direction dfSet or gramSet
        while j < len(traceList) - 1 and traceList[j + 1] + traceList[j] in dfSet:
            if traceList[j] + traceList[j+1] + traceList[j] not in gramSet and traceList[j+1] + traceList[j] + traceList[j+1] not in gramSet:
                #print(traceList[j + 1] + traceList[j])
                j +=1
                count += 1
                #print('count ', count)
            else:
                break
        
        if count > 1:
            traceList = join_and_replace(traceList,i,count)
            #print(traceList)
            #print('i=',i)
            #i = j + count -2
            
            #print(len(traceList))
        i += 1
    return traceList

x = ['O','L','P','O','L','R']
dfRel = {'PL', 'OP', 'RL'}
threeGram = {'OLP', 'LPL','POL', 'OLR'}
            
checkPara(x,dfRel,threeGram)

['O', 'L', 'OP', 'LR']

In [31]:
#Generate for each trace the trace including sets of parallel activties
logVar["paraList"] = logVar["charList"].apply(lambda x: checkPara(x,df_relations,gram_relations))

In [32]:
from IPython.display import display
#display(df_relations)
display(list(logVar["paraList"]))

[['Þ',
  'cªÄÅŕŗżƋƽ',
  'áĀĄ',
  "\x05\x0c\x1c'QUs\x98¦§©ÌóŌŜŦƊƞ",
  '"<\x89\x8d¤ÔŁƺ',
  '\x14h\x90\x9fÝĿņ',
  '\x17ÊłƎư',
  'ĝŸ',
  '\x15]\x99\x9eďĨƔƩ',
  'Œ'],
 ['ÄÅŕŗƽ',
  'ż',
  "\x1c'c\x98§©ªÞĀĄŌŜƋƞ",
  '\x05\x0c"<QUs\x8d¤¦ÌÔÝáóŁŦƊƺ',
  '\x89',
  '\x14h\x92\x9fĞņ',
  'Ơ',
  '\x94',
  '\x16\\¡ŵ',
  'Ć',
  'ďĕƪ',
  'ñ',
  '\xa0Ĕĺ',
  '\x149\x90\x91ĿƁ',
  '+',
  'Ã',
  'ū',
  'Î',
  'V',
  '÷',
  '\x0fś',
  'Ɠ',
  'ð',
  '\x99ļłŸƔƯ',
  '\x15]\x9eĨŒƩ'],
 ['Éìŗƽ',
  '\x1ccÐĄŮ',
  'Ç',
  'Ìť',
  '\x05\x0c"<\x89\x8d¤ÔŀƊƺ',
  '\x14$9h\x91\x9fļĿŔ',
  ']\x99\x9eĎĕĨŸƔ',
  'Œ'],
 ['ŗ',
  'Þ',
  'ÅƋ',
  'ªƽ',
  'cÄż',
  '§ŌŦƞ',
  '"Ł',
  'QUsƊ',
  "'\x98¦ÝóĄŜ",
  '\x1c¤',
  '\x0c©ÌÔ',
  'Ā',
  '\x8dá',
  '\x05<ƺ',
  '\x9fĞ',
  '\x92',
  '\x94',
  '\x89',
  '\x14hņ',
  '\x16\\¡ŵ',
  '\x149\x90\x91\xa0ĔĺĿƁ',
  'Ơ',
  'Ć',
  '+',
  'Ã',
  'ū',
  'Î',
  'V',
  '÷',
  '\x0fś',
  'Ɠ',
  '\x99ďłŸƔƯ',
  'ñ',
  'ð',
  '\x15]\x9eĨƩƪ',
  'Œ'],
 ['Yƽ',
  'cÐŮ',
  '\x1c',
  'ŝ',
  'Ìť',
  'Ą',
  '\x05\x0c"

## Graph based features

In [33]:
# 1. transfer strings (string numbers) to integers --> HERE we need a new encoding!

#logVar = logVar.astype({"trace_variant": str}) #--> only needed if list values were transformed to int at one point

def intEncoder(character_List):
    return [np.where(np.array(list(dict.fromkeys(character_List)))==e)[0][0]for e in character_List]

logVar["intList"] = logVar["paraList"].apply(lambda x: intEncoder(x))

In [34]:
# 2. transfer intList to int_tupleList

#Create tuple lists
def tuple_list(list_of_encodedActivities):
    #list.insert(0, '*')
    #list.append('*')
    list_new = []
    last_element = list_of_encodedActivities[-1]
    for i in range(len(list_of_encodedActivities)):
        new = tuple(list_of_encodedActivities[i:i+2])
        list_new.append(new)
    del list_new[-1]
    if list_of_encodedActivities.count(last_element) == 1: #check wether last activity in trace has some adjancency relation
        list_new.append((last_element,)) ### NOT Correct
    return list_new

#q = [0,0,0,0,1,1,2,3,4,5,3,2,4,0,5,6]
#tuple_list(q)

logVar["int_tupleList"] = logVar["intList"].apply(lambda x: tuple_list(x))
#logVar["int_tupleList"]

In [35]:
# 3. generate Adjacency List

def adj_list(list_of_tuples):
    adj_list_new = {}
    try:
        for node1, node2 in list_of_tuples:
            #print(node1, node2)
            if node1 not in adj_list_new:
                newlist = []
                newlist.append(node2)
                adj_list_new[node1] = newlist
                #print(adj_list3)
        
            else:
                if node2 not in adj_list_new[node1]:
                    #mylist.extend(adj_list3[node1])
                    adj_list_new[node1].append(node2)
                    #print(adj_list3)
                    #adj_list3[node1] = mylist
    
    #in case activity has no adjacent activity - only possible for last activity --> tuple: (lastAct,)
    except ValueError as ve:
        lastValue = list_of_tuples[-1][0] 
        adj_list_new[lastValue] = list()
    return list(adj_list_new.values())

#q = [0,0,0,0,1,1,2,3,4,5,3,2,4,0,5,6]
#l = tuple_list(q)
#adj_list(l)

logVar["int_adjList"] = logVar["int_tupleList"].apply(lambda x: adj_list(x))
#logVar["int_adjList"]

In [36]:
from collections import deque

def bfs_4(graph, start, end):
    
    graph = {v: k for v, k in enumerate(graph)}
    #print(start, end)
    queue = deque([(start, 0)])
    seen = set()
    while queue:
        #print(queue)
        node, distance = queue.popleft()
        #if not node:
            #print(start, end, queue)
            #print("GRAPH LIST", graph)
        if node in seen:
            continue
        seen.add(node)
        if node == end:
            return distance 
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
        
#x = {0: [0, 1], 1: [2, 1, 0, int], 2:[2], [3: [1, 5, 3, 7], 4: [3], 5: [6, 5], 6: [1, 7], 7: [8, 9, 7], 8: [5, 8, 10], 9: [3]}
#y = [[0, 1, 5], [1, 2], [3, 4], [4, 2], [5, 0], [3, 6], []]
#bfs_4(y, 1, 6)

In [37]:
from collections import defaultdict, deque

def reverse_graph(graph):
    reversed_graph = defaultdict(list)
    for node in graph:
        for neighbor in graph[node]:
            reversed_graph[neighbor].append(node)
    return reversed_graph


def bfs_5(graph, start, end):
    queue = deque([(start, 0)])
    seen = set()
    visited = {}
    while queue:
        node, distance = queue.popleft()
        if node in seen:
            continue
        seen.add(node)
        if node == end: # maybe quicker if adjacent directly checked
            return visited
        for adjacent in graph.get(node, []):
            queue.append((adjacent, distance + 1))
            if adjacent not in visited:
                visited.update({adjacent:distance})

            
def common_ancestors(graph, node1, node2): 
    #remove cross type edge between node1 and node2
    graph[node1].remove(node2) #maybe issue, if graph is changed and not copy of graph?
    graph = {v: k for v, k in enumerate(graph)}
    graphReverse = reverse_graph(graph)
    setNode1 = bfs_5(graphReverse, node1, 0)
    setNode2 = bfs_5(graphReverse, node2, 0)
    if next((a for a in list(setNode1) if a in list(setNode2)), None) == None:
        firstCommonAnces = next((a for a in list(setNode2) if a in list(setNode1)), None)
    else:
        firstCommonAnces = next((a for a in list(setNode1) if a in list(setNode2)), 0)
    
    #uses a hash map to identify the first common ancestor in both lists
    #looks for the first common ancestor in setNode1, which can also be found in setNode2 
    #--> this might not be the closest distance between setNode1 and setNode2
    #--> e.g., for x= [0,1,3,7,5,6] and y= [4,5,7,8,3] 7 might be closest ancestor, although algo detects 3 !
    #distance = setNode1[firstCommonAnces] + setNode2[firstCommonAnces]
    
    
    if firstCommonAnces != None:   # ISSUE: in some cases the firstCommonAnces cannot be detected!
        ancesDistNode1 =  setNode1[firstCommonAnces] + 1 #the edge from node1 to first parent is counted as 0 by algorithm, therefore +1
        ancesDistNode2 =  setNode2[firstCommonAnces] + 1
        numberSkips = abs(ancesDistNode1 - ancesDistNode2)
        numberCross = min(ancesDistNode1, ancesDistNode2)
    else:
        numberSkips, numberCross = (0,1)
    return numberSkips, numberCross
    #if all(x in crossType for x in i):
    

    

#graphList = [[1], [2, 4, 1], [3, 2, 1], [], [5, 4], [5, 4, 6], [7], []]
#c = [[1, 4], [2], [3], [0, 5], [3, 5], []]
#c2 = {v: k for v, k in enumerate(c)}
#common_ancestors(c, 4, 5)
#reverse_graph(c2)

In [38]:
#Create List for decoding traces
from collections import OrderedDict
logVar["indexList"] = logVar["paraList"].apply(lambda x: list(OrderedDict.fromkeys(x)))

### Cosine Edge Type + length (no df relations)

In [39]:
class Graph1:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = [['sequ', 1]]
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return self.structural_array
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                self.structural_array[0][1] += 0
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append(['1back ',1])
                    #self.structural_array.append(['back ',1])
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append(['2back ',2])
                    #self.structural_array.append(['back ',2])
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append([str(x+1)+'back ',x+1])
                    #self.structural_array.append(['back ',x+1])
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append([str(y-1)+'forward ',y-1])
                #self.structural_array.append(['forward ' ,y-1])
                #self.structural_array.extend((y-1)*['forward']) # -1 to exclude one edge: (A:B,C;B:C;C:[]) ...the dist A --> C is 2 without forward edge, but we are skipping only one activity
                #self.structural_array.append(str(y-1)+'f')
                
            else:
                #Possibly first check, whether two nodes connected by cross-type have identical parent
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                #self.structural_array.append(['forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]),numberSkips])
                self.structural_array.append([str(numberCross)+'cross ',numberCross])

    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1


In [40]:
from collections import Counter

def transform_list_of_pairs(pairs):
    return [pair[0] for pair in pairs]



def count_entries(input_list):
    # Count the occurrences of each unique entry in the list
    counter = Counter(input_list)
    
    # Create a NumPy array from the counter dictionary
    result = np.array([[key, count] for key, count in counter.items()], dtype=object)
    
    return result

# Example usage
#input_list = ['sequ', '2back', '2back']
#result = count_entries(input_list)
#print(result)

logVar["int_strucLengthList2"] = logVar.apply(lambda x: Graph1(x.int_adjList, x.indexList).dfs(), axis =1)
#logVar["relFrequVec1"] = logVar["int_strucLengthList2"].apply(lambda x: transform_list_of_pairs(x))
#logVar["relFrequVec1"] = logVar["relFrequVec1"].apply(lambda x: count_entries(x))

In [ ]:
###### Changed set of high-level features

In [41]:
# First generate list of parallel constructs with # of activities involved

a = logVar['paraList'][0]
print(a)


        
def para_features(para_list):
    paraAct = []
    for i in range(len(para_list)):
        if len(para_list[i]) > 1:
            paraAct.append([str(len(para_list[i])) + 'para', len(para_list[i])])
    return paraAct


para_features(a)   

['Þ', 'cªÄÅŕŗżƋƽ', 'áĀĄ', "\x05\x0c\x1c'QUs\x98¦§©ÌóŌŜŦƊƞ", '"<\x89\x8d¤ÔŁƺ', '\x14h\x90\x9fÝĿņ', '\x17ÊłƎư', 'ĝŸ', '\x15]\x99\x9eďĨƔƩ', 'Œ']


[['9para', 9],
 ['3para', 3],
 ['18para', 18],
 ['8para', 8],
 ['7para', 7],
 ['5para', 5],
 ['2para', 2],
 ['8para', 8]]

In [42]:
logVar["int_strucLengthList4"] = logVar["paraList"].apply(lambda x: para_features(x))
logVar["int_strucLengthList5"] = logVar["int_strucLengthList2"] + logVar["int_strucLengthList4"]
logVar["int_strucLengthList5"] = logVar["int_strucLengthList5"].apply(lambda x: np.array(x))
logVar["relFrequVec1"] = logVar["int_strucLengthList5"].apply(lambda x: transform_list_of_pairs(x))
logVar["relFrequVec1"] = logVar["relFrequVec1"].apply(lambda x: count_entries(x))

In [201]:
#Cosine distance based on edge types and parallel activities
cos_graph_dis = matrix_calc(logVar["relFrequVec1"],cosineDist)
results(cos_graph_dis)

NN:   0.2257142857142857
P@10: 0.24485714285714288
Tri:  tensor(0.4999)
Sil:  -0.06977650502069445


In [202]:
#Cosine distance based on edge types
#cos1_dis = matrix_calc(logVar["1-gram"],cosineDist)
#cos2_dis = matrix_calc(logVar["2-gram"],cosineDist)

agg_cos = cos1_dis + cos2_dis + cos_graph_dis
results(agg_cos)

NN:   0.5628571428571428
P@10: 0.3602857142857143
Tri:  tensor(0.5260)
Sil:  -0.04141911847763973


In [ ]:
# Old results without parallel activities

In [47]:
#Cosine distance based on edge types
#cos_graph_dis = matrix_calc(logVar["relFrequVec1"],cosineDist)
#results(cos_graph_dis)

NN:   0.20571428571428568
P@10: 0.1974285714285714
Tri:  tensor(0.1517)
Sil:  -0.25771454110379977


In [48]:
#Cosine distance based on edge types
#agg_cos = cos1_dis + cos2_dis + cos_graph_dis
#results(agg_cos)

NN:   0.5942857142857143
P@10: 0.4145714285714286
Tri:  tensor(0.5236)
Sil:  -0.044747027287532264


### Jaccard Edge Type and length + df relation

In [45]:
class Graph2:
    # instance variables
    def __init__(self, graph_list2, indexList):
        # v is the number of nodes/vertices
        self.time = 0
        self.traversal_array = []
        self.structural_array = []
        #self.structural_array = []
        self.graph_list = graph_list2
        self.v = len(graph_list2)
        self.indexList = indexList

    # function for dfs
    def dfs(self):
        self.start_time = [-1]*self.v
        self.end_time = [-1]*self.v
 
        for node in range(self.v):
            if self.start_time[node] == -1:
                self.traverse_dfs(node)
                
        return self.structural_array
        #print()
        #print("DFS Traversal: ", self.traversal_array)
        #print()
 
    def traverse_dfs(self, node):
        self.traversal_array.append(node)
        # get the starting time
        self.start_time[node] = self.time
        self.time += 1
        # traverse through the neighbours
        for neighbour in self.graph_list[node]:

            # when the neighbor was not yet visited
            if self.start_time[neighbour] == -1:                
                #self.structural_array[0][1] += 0
                #self.structural_array.append('tree ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ')
                self.structural_array.append('tree')
                self.traverse_dfs(neighbour)
                
            # otherwise when the neighbour's visit is still ongoing:
            elif self.end_time[neighbour] == -1:
                if node == neighbour:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(1))
                    #self.structural_array.append([str(1)+'b'])
                
                elif node in self.graph_list[neighbour]:
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(2))
                    #self.structural_array.append(str(2)+'b')
                    
                else:
                    x = bfs_4(self.graph_list, neighbour, node)
                    self.structural_array.append('back ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(x+1))
                    #self.structural_array.append(str(x+1)+'b')
                
            # otherwise when the neighbour's visit started before the current node's visit:
            elif self.start_time[node] < self.start_time[neighbour]:
                graph_list_copy = copy.deepcopy(self.graph_list)
                graph_list_copy[node].remove(neighbour)
                y = bfs_4(graph_list_copy, node, neighbour)
                self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(y-1))
                #self.structural_array.extend((y-1)*['forward']) # -1 to exclude one edge: (A:B,C;B:C;C:[]) ...the dist A --> C is 2 without forward edge, but we are skipping only one activity
                #self.structural_array.append(str(y-1)+'f')
                
            else:
                #Possibly first check, whether two nodes connected by cross-type have identical parent
                numberSkips, numberCross = common_ancestors(self.graph_list, node, neighbour)
                #self.structural_array.append('forward ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberSkips))
                self.structural_array.append('cross ' + str(self.indexList[node]) + ' ' + str(self.indexList[neighbour]) + ' ' + str(numberCross))
                #self.structural_array.append(str(numberSkips)+'f')
                #self.structural_array.append(str(numberCross)+'c')
    
        # Indentation corrected:
        self.end_time[node] = self.time
        self.time += 1

In [46]:
logVar["int_strucLengthList3"] = logVar.apply(lambda x: Graph2(x.int_adjList, x.indexList).dfs(), axis =1)
#logVar

In [47]:
# Generate list of parallel constructs with # of activities involved

a = logVar['paraList'][0]
print(a)


        
def para_features2(para_list):
    paraAct = []
    for i in range(len(para_list)):
        if len(para_list[i]) > 1:
            paraAct.append('para ' + para_list[i])
    return paraAct


para_features2(a) 

['Þ', 'cªÄÅŕŗżƋƽ', 'áĀĄ', "\x05\x0c\x1c'QUs\x98¦§©ÌóŌŜŦƊƞ", '"<\x89\x8d¤ÔŁƺ', '\x14h\x90\x9fÝĿņ', '\x17ÊłƎư', 'ĝŸ', '\x15]\x99\x9eďĨƔƩ', 'Œ']


['para cªÄÅŕŗżƋƽ',
 'para áĀĄ',
 "para \x05\x0c\x1c'QUs\x98¦§©ÌóŌŜŦƊƞ",
 'para "<\x89\x8d¤ÔŁƺ',
 'para \x14h\x90\x9fÝĿņ',
 'para \x17ÊłƎư',
 'para ĝŸ',
 'para \x15]\x99\x9eďĨƔƩ']

In [48]:
logVar["int_strucLengthList6"] = logVar["paraList"].apply(lambda x: para_features2(x))
logVar["int_strucLengthList7"] = logVar["int_strucLengthList3"] + logVar["int_strucLengthList6"]

In [208]:
#Jacc similarity based on edge types

jacc_graph = matrix_calc(logVar["int_strucLengthList7"],jaccard_similarity)
results(jacc_graph)

NN:   0.4714285714285714
P@10: 0.3274285714285714
Tri:  tensor(0.4684)
Sil:  -0.012122561995200185


In [209]:
#Jacc sim based on edge types
#Jacc1_dis = matrix_calc(logVar["charList"],jaccard_similarity)
#Jacc2_dis = matrix_calc(logVar["dfList"],jaccard_similarity)

agg_jacc = Jacc1_dis + Jacc2_dis + jacc_graph
results(agg_cos)

NN:   0.5628571428571428
P@10: 0.3602857142857143
Tri:  tensor(0.5260)
Sil:  -0.04141911847763973


In [ ]:
# OLD

In [62]:
#Jacc similarity based on edge types

jacc_graph = matrix_calc(logVar["int_strucLengthList3"],jaccard_similarity)
results(jacc_graph)

NN:   0.21999999999999992
P@10: 0.20314285714285715
Tri:  tensor(0.1529)
Sil:  -0.23659391430012866


In [63]:
#Jacc sim based on edge types
agg_jacc = Jacc1_dis + Jacc2_dis + jacc_graph
results(agg_cos)

NN:   0.5942857142857143
P@10: 0.4145714285714286
Tri:  tensor(0.5236)
Sil:  -0.044747027287532264


## Eventually Follows

In [64]:
#Spatial distance between strings


from scipy.spatial import distance


def distanceSpatial(traceString, char1, char2):
    positions_letter1 = [pos for pos, char in enumerate(traceString) if char == char1]
    positions_letter2 = [pos for pos, char in enumerate(traceString) if char == char2]
    
    distList = []
    

    for i in range(len(positions_letter1)):
        for j in range(len(positions_letter2)):
            dist = positions_letter2[j] - positions_letter1[i]
            if dist > 0:
                    #print(dist)
                distList.append(dist)
                    
    
    if not distList: #distList.append(0) #in the case the char1 is after char2 asign dist 0, i.e. char2 cannot be reached from char1
        return 0
    else:
        return 1/min(distList)





def commonDistance(trace1, trace2):
    
    commonSet = set(trace1) & set(trace2)

    commonList = list(commonSet)
    commonList.sort()
    #print(commonList)

    n = len(commonSet)
    dist_matrix1 = np.zeros((n,n))
    dist_matrix2 = np.zeros((n,n))

    for i in range(n):
        for j in range(i, n):
            dist_matrix1[i,j] = distanceSpatial(trace1, commonList[i], commonList[j])
        
    for i in range(n):
        for j in range(i, n):
            dist_matrix2[i,j] = distanceSpatial(trace2, commonList[i], commonList[j])
    
    #print(dist_matrix1, dist_matrix2)
    return distance.cosine(dist_matrix1.ravel(), dist_matrix2.ravel())



#x = 'ABCDEF'
#y = 'ABCDEBCDEBCDEF'
#z = 'ABCDEBCDEF'
#print(dist_matrix)
#distanceSpatial(x, 'A', 'E')
#listVec = logVar["strings"]
#x= listVec[0]
#y= listVec[1]
#commonDistance(x, y)

In [65]:
dist_matrix_evFollows = matrix_calc(logVar["strings"],commonDistance)
dist_matrix_evFollows = np.nan_to_num(dist_matrix_evFollows, nan=0)
agg_evFollows = 0.7*dist_matrix_evFollows + 0.3*cos1_dis 
results(agg_evFollows)

C:\Users\la1949\AppData\Local\anaconda3\envs\ProcessMining\Lib\site-packages\scipy\spatial\distance.py:685: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


NN:   0.45714285714285713
P@10: 0.35914285714285715
Tri:  tensor(0.5252)
Sil:  -0.044154845739131934


## Maximal Repeat

In [66]:
from suffix_tree import Tree

In [67]:
#tree.maximal_repeats
def maxRepeat(tree):
    mrList=[]
    for C, path in sorted(tree.maximal_repeats()):
        mrList.append(str(path))
    return mrList

#test_tree = Tree({"A": "aaacdcdcbedbccbadbdebdc"})
#maxRepeat(test_tree)

In [68]:
#create vector based on maximal repeats
logVar["mrList"] = logVar["strings"].apply(lambda x: maxRepeat(Tree({"A": x})))
logVar["mrVector"] = logVar["mrList"].apply(lambda x: createVector(tuple(x)))

In [71]:
#Cosine distance based on maxR

cos_mr = matrix_calc(logVar["mrVector"],cosineDist)
cos_mr = np.nan_to_num(cos_mr, nan=0)
results(cos_mr)

NN:   0.2
P@10: 0.2
Tri:  tensor(0.0193)
Sil:  -0.2479136835528282


In [72]:
#Euclidean distance based on maxR

euc_mr = matrix_calc(logVar["mrVector"],euclidDist)
euc_mr = np.nan_to_num(euc_mr, nan=0)
results(euc_mr)

NN:   0.20285714285714285
P@10: 0.1985714285714286
Tri:  tensor(0.1490)
Sil:  -0.26468885883084137


In [73]:
#Jaccard similarity based on maxR

jacc_mr = matrix_calc(logVar["mrList"],jaccard_similarity)
results(jacc_mr)

NN:   0.19428571428571428
P@10: 0.198
Tri:  tensor(0.1325)
Sil:  -0.18014079859510926


# Trace2Vec

In [223]:
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [228]:
logVar["words"] = logVar["c:n_chr"].apply(lambda x: ' '.join(x))
logVar["words"][0]

'Þ ƽ Å Ƌ ª Ä ŕ ż c ŗ Ā Ą á Ō § ƞ \x1c © \' Ŝ \x98 Q ¦ Ŧ Ì U ó \x0c Ɗ s \x05 " ƺ < Ô ¤ Ł \x8d \x89 h ņ \x9f \x14 Ŀ \x90 Ý ł Ê ư \x17 Ǝ Ÿ ĝ \x99 ď Ɣ Ʃ \x15 ] \x9e Ĩ Œ'

In [229]:
documents = list(logVar['words'])

# Step 1: Tokenize documents
texts = [
    [word for word in document.lower().split()]
    for document in documents
]

# Step 2: Tag the documents

from gensim.models.doc2vec import TaggedDocument

tagged_docs = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(documents)]

In [231]:
#Derive and evaluate model

from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity

def trainModel(documents,vec_size):
    
    # Step 3: Initialize and train the model
    model = Doc2Vec(vector_size=vec_size, window=3, min_count=0.0, min_alpha=0.025, epochs=10, dm = 0, alpha=0.025)
    model.build_vocab(documents)
    model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)
    
    # Step 4: infer vectors for each document
    vectors = [model.infer_vector(doc) for doc in texts]
    
    # Step 5: calculate cosine distances
    cos = cosine_similarity(vectors)
    
    return results(cos)

In [232]:
trainModel(tagged_docs,16)

NN:   0.17714285714285713
P@10: 0.18
Tri:  tensor(0.4669)
Sil:  -0.069205634


In [233]:
trainModel(tagged_docs,32)

NN:   0.18285714285714286
P@10: 0.18485714285714286
Tri:  tensor(0.4690)
Sil:  -0.07415957


In [234]:
trainModel(tagged_docs,64)

NN:   0.19714285714285715
P@10: 0.1857142857142857
Tri:  tensor(0.4694)
Sil:  -0.07714858


In [235]:
trainModel(tagged_docs,128)

NN:   0.2
P@10: 0.19342857142857142
Tri:  tensor(0.4728)
Sil:  -0.07953263
